In [11]:
# import Libraries
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown, update_display
import gradio as gr

In [12]:
# setup environment
MODEL = "gpt-4o-mini"

load_dotenv(override=True)
openai = OpenAI()

if openai.api_key is None:
    raise ValueError("OpenAI API key not found")
else:
    print("OpenAI API key found")

OpenAI API key found


In [ ]:
system_prompt = "You are a helpful assistant that responds in markdown format without code blocks."

def message_gpt(message, history):
    # Adds system prompt to the messages list
    messages = [{"role": "system", "content": system_prompt}]
    
    # Adds previous messages to the messages list
    for turn in history:
        # Check if turn is actually a list/tuple and has at least 2 elements (user, AI)
        if isinstance(turn, (list, tuple)) and len(turn) >= 2:
            user_message = turn[0]
            assistant_message = turn[1]
            messages.append({"role": "user", "content": str(user_message)})
            messages.append({"role": "assistant", "content": str(assistant_message)})
    
    # Adds current user message to the messages list
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(
        model=MODEL,
        messages = messages,
        stream=True
    )
    results = ""
    for chunk in stream:
        # Check if content is not None before adding
        if chunk.choices[0].delta.content:
            results += chunk.choices[0].delta.content
            yield results

In [14]:
# message_input = gr.Textbox(label="Your Message: ", info="Enter your message here", lines=7)
# message_output = gr.Markdown(label="Response: ")

# Create a chat interface 
view = gr.ChatInterface(
    fn=message_gpt, 
    title="Chat with GPT", 
    description="Enter your message and get a response from GPT",
    examples=[
        "Hello, how are you?",
        "Explain the Transformer architecture",
        "What is the meaning of life?",
    ],
    flagging_mode="never"
)
view.launch()


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
